In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pickle

In [40]:
def chooseModel(province,inputshape):
    if(province=='BKK'):
        modelLSTM_BKK = Sequential()

        modelLSTM_BKK.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 80, activation = 'relu' ,return_sequences = True))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_BKK.add(Dropout(0.2))


        modelLSTM_BKK.add(Dense(units = 12))
        return modelLSTM_BKK
    if(province=='Chiangmai'):
        modelLSTM_Chiangmai = Sequential()

        modelLSTM_Chiangmai.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Chiangmai.add(Dropout(0.2))
        modelLSTM_Chiangmai.add(LSTM(units = 80, activation = 'relu'))
        modelLSTM_Chiangmai.add(Dropout(0.2))


        modelLSTM_Chiangmai.add(Dense(units = 12))
        return modelLSTM_Chiangmai
    if(province=='Khonkaen'):
        modelLSTM = Sequential()

        modelLSTM.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM.add(Dropout(0.2))

        # modelLSTM.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
        # modelLSTM.add(Dropout(0.2))

        modelLSTM.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM.add(Dropout(0.2))

        modelLSTM.add(Dense(units = 12))
        return modelLSTM
    if(province=='Rayong'):
        modelLSTM_Rayong = Sequential()

        modelLSTM_Rayong.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Rayong.add(Dropout(0.2))

        modelLSTM_Rayong.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Rayong.add(Dropout(0.2))


        modelLSTM_Rayong.add(Dense(units = 12))
        return modelLSTM_Rayong

    if(province=='Saraburi'):
        modelLSTM_Saraburi = Sequential()

        modelLSTM_Saraburi.add(LSTM(units = 120, activation = 'relu',  input_shape = (inputshape, 4)))
        modelLSTM_Saraburi.add(Dropout(0.2))

        modelLSTM_Saraburi.add(Dense(units = 12))
        return modelLSTM_Saraburi
    if(province=='Surat'):
        modelLSTM_Surat = Sequential()

        modelLSTM_Surat.add(LSTM(units = 80, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Surat.add(Dropout(0.2))
        # modelLSTM_Surat.add(LSTM(units = 120, activation = 'relu', return_sequences = True))
        # modelLSTM_Surat.add(Dropout(0.2))
        modelLSTM_Surat.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Surat.add(Dropout(0.2))



        modelLSTM_Surat.add(Dense(units = 12))

        return modelLSTM_Surat

In [41]:
all_config ={'BKK':{
                    'batch': 32,
                    'epoch': 10,
},
'Chiangmai':{
                    'batch': 32,
                    'epoch': 10,
},
'Khonkaen':{
                    'batch': 32,
                    'epoch': 10,
}
,'Rayong':{
                    'batch': 32,
                    'epoch': 10,
}
,'Saraburi':{
                    'batch': 32,
                    'epoch': 10,
},'Surat':{
                    'batch': 64,
                    'epoch': 10,
},}

In [42]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
# provinces = ['Surat']

for province in provinces[3:]:
    print(province)
    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)

    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)

    data_test = pd.read_csv(province+'_clean.csv', date_parser = True)

    scaler = MinMaxScaler()
    data_training = scaler.fit_transform(data_training)
    scaler.inverse_transform(data_training)
    scale = 1/scaler.scale_

    X_train = []
    y_train = []

    for i in range(60, data_training.shape[0]):
        X_train.append(data_training[i-60:i])
        a = []
        for j in range(1,13):
            #print(i,j,i-60+j*6)
            a.append(data_training[min(i+j*6,data_training.shape[0]-1),0])
        y_train.append(a)


    X_train, y_train = np.array(X_train), np.array(y_train)



        #create callback
    filepath = province+'_modelLSTM.hdf5'
    checkpoint = ModelCheckpoint(filepath=filepath, 
                                monitor='loss',
                                verbose=0, 
                                save_best_only=True,
                                mode='min')

    earlystopping = EarlyStopping(
        monitor='loss', 
        patience=3, 
        min_delta=0, 
        mode='auto'
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='loss', 
        factor=0.2,   
        patience=2, 
        min_lr=0.001,
        verbose=0
    )
    callbacks = [checkpoint, earlystopping, reduce_lr]

    modelLSTM = chooseModel(province,X_train.shape[1])
    modelLSTM.compile(optimizer='adam', loss = 'mean_squared_error')
    history = modelLSTM.fit(X_train, y_train, epochs=all_config[province]['epoch'], batch_size=all_config[province]['batch'], callbacks=callbacks)

    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)
    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)
    data_test = data_test.drop(['Unnamed: 0', 'lat', 'long','date_time'], axis = 1)

    past_60_days = data_training.tail(60)
    df = past_60_days.append(data_test, ignore_index = True)
    
    df.fillna(method='ffill', inplace=True)

    inputs = scaler.transform(df)
    
    X_test = []
    y_test = []

    for i in range(60, inputs.shape[0]):
        X_test.append(inputs[i-60:i])
        a = []
        for j in range(1,13):
            #print(i,j,i-60+j*6)
            a.append(inputs[min(i+j*6,inputs.shape[0]-1),0])
        y_test.append(a)
    X_test, y_test = np.array(X_test), np.array(y_test)
    y_pred = modelLSTM.predict(X_test)
    scale = 1/scaler.scale_[0]
    y_pred = y_pred*scale
    
    df1 = pd.DataFrame(y_pred)
    df1.to_csv(province+'_pred.csv',index=False)

    with open('lstm_'+province.lower()+'.pickle', 'wb') as fp:
        pickle.dump(y_pred, fp)

    


Rayong
Epoch 1/10
821/821 [==============================] - 80s 95ms/step - loss: 0.0063
Epoch 2/10
821/821 [==============================] - 64s 78ms/step - loss: 0.0057
Epoch 3/10
821/821 [==============================] - 60s 73ms/step - loss: 0.0054
Epoch 4/10
821/821 [==============================] - 61s 74ms/step - loss: 0.0051
Epoch 5/10
821/821 [==============================] - 67s 81ms/step - loss: 0.0049
Epoch 6/10
821/821 [==============================] - 59s 72ms/step - loss: 0.0047
Epoch 7/10
821/821 [==============================] - 57s 70ms/step - loss: 0.0045
Epoch 8/10
821/821 [==============================] - 61s 74ms/step - loss: 0.0043
Epoch 9/10
821/821 [==============================] - 61s 75ms/step - loss: 0.0042
Epoch 10/10
821/821 [==============================] - 62s 75ms/step - loss: 0.0041
Saraburi
Epoch 1/10
821/821 [==============================] - 37s 44ms/step - loss: 0.0021
Epoch 2/10
821/821 [==============================] - 34s 41ms/step - 

In [43]:
df1.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [44]:
y_pred

array([[23.470444, 19.99715 , 15.325587, ..., 17.893097, 12.586964,
        17.62326 ],
       [23.396534, 18.075052, 15.749897, ..., 16.218534, 13.266329,
        17.913681],
       [23.198404, 16.383413, 16.410107, ..., 14.711015, 14.112244,
        18.1171  ],
       ...,
       [18.349537, 17.579723,  9.85421 , ..., 18.543331, 10.507658,
        15.199989],
       [18.728148, 16.254126,  9.995404, ..., 17.268074, 10.604116,
        16.160423],
       [19.084887, 14.522037, 10.688286, ..., 15.588583, 11.332602,
        16.91915 ]], dtype=float32)